>>># iNeuron.ai
internship project (adult census income prediction)

In [63]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
import pickle

In [55]:
df=pd.read_csv('adult.csv')
df.head()

age          workclass  fnlwgt   education  education-num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital-status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital-gain  capital-loss  hours-per-week         country  salary  
0          2174             0              40   United-States   <=50K  
1             0             0              13   United-States   <=50K  
2             0             0              40   United-States   <=50K  
3             0             0              40   United-States   <=50K  
4             0             0              40            Cuba   <=50K

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  country         32561 non-null  object
 14  salary          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [15]:
df[['workclass']].value_counts()

workclass        
 Private             22696
 Self-emp-not-inc     2541
 Local-gov            2093
 ?                    1836
 State-gov            1298
 Self-emp-inc         1116
 Federal-gov           960
 Without-pay            14
 Never-worked            7
dtype: int64

In [22]:
df.describe()

age        fnlwgt  education-num  capital-gain  capital-loss  \
count  32561.000000  3.256100e+04   32561.000000  32561.000000  32561.000000   
mean      38.581647  1.897784e+05      10.080679   1077.648844     87.303830   
std       13.640433  1.055500e+05       2.572720   7385.292085    402.960219   
min       17.000000  1.228500e+04       1.000000      0.000000      0.000000   
25%       28.000000  1.178270e+05       9.000000      0.000000      0.000000   
50%       37.000000  1.783560e+05      10.000000      0.000000      0.000000   
75%       48.000000  2.370510e+05      12.000000      0.000000      0.000000   
max       90.000000  1.484705e+06      16.000000  99999.000000   4356.000000   

       hours-per-week  
count    32561.000000  
mean        40.437456  
std         12.347429  
min          1.000000  
25%         40.000000  
50%         40.000000  
75%         45.000000  
max         99.000000

In [26]:
df.isnull().sum().sum()

0

In [38]:
df.select_dtypes(exclude=['int64']).head()

workclass   education       marital-status          occupation  \
0          State-gov   Bachelors        Never-married        Adm-clerical   
1   Self-emp-not-inc   Bachelors   Married-civ-spouse     Exec-managerial   
2            Private     HS-grad             Divorced   Handlers-cleaners   
3            Private        11th   Married-civ-spouse   Handlers-cleaners   
4            Private   Bachelors   Married-civ-spouse      Prof-specialty   

     relationship    race      sex         country  salary  
0   Not-in-family   White     Male   United-States   <=50K  
1         Husband   White     Male   United-States   <=50K  
2   Not-in-family   White     Male   United-States   <=50K  
3         Husband   Black     Male   United-States   <=50K  
4            Wife   Black   Female            Cuba   <=50K

In [61]:
df['country'].nunique()
# there are  42 country available

42

In [79]:
df[['country']].value_counts().tail()

country                    
 Outlying-US(Guam-USVI-etc)    14
 Hungary                       13
 Honduras                      13
 Scotland                      12
 Holand-Netherlands             1
dtype: int64

In [88]:
df[['salary']].value_counts().shape

(2,)

In [104]:
ls=df.select_dtypes(exclude=('int64')).keys()
ls

Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'country', 'salary'],
      dtype='object')

In [110]:
df[['workclass']].value_counts()

workclass        
 Private             22696
 Self-emp-not-inc     2541
 Local-gov            2093
 ?                    1836
 State-gov            1298
 Self-emp-inc         1116
 Federal-gov           960
 Without-pay            14
 Never-worked            7
dtype: int64

In [69]:
df['salary'].value_counts()

 <=50K    24720
 >50K      7841
Name: salary, dtype: int64

In [59]:
df['result']=df['salary'].map(len)
# converting result feature in string lenght

In [62]:
df[['result']].value_counts()

result
6         24720
5          7841
dtype: int64

In [65]:
d={6:1,5:0}
def mapmy(x):
    return d[x]

In [66]:
df['result']=df['result'].map(mapmy)

In [70]:
df['result'].value_counts()
# according to string length i replaced value by 0 and 1

1    24720
0     7841
Name: result, dtype: int64

In [74]:
ndf=df.drop('salary',axis='columns')
ndf.head(2)
# finally cleaned this column so i am removing it.

age          workclass  fnlwgt   education  education-num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   

        marital-status        occupation    relationship    race    sex  \
0        Never-married      Adm-clerical   Not-in-family   White   Male   
1   Married-civ-spouse   Exec-managerial         Husband   White   Male   

   capital-gain  capital-loss  hours-per-week         country  result  
0          2174             0              40   United-States       1  
1             0             0              13   United-States       1

In [10]:
len(list(df['workclass'].unique()))
# there are only 9 classes

9

In [15]:
df[['workclass']].value_counts().keys()
# names of 9 classes

MultiIndex([(         ' Private',),
            (' Self-emp-not-inc',),
            (       ' Local-gov',),
            (               ' ?',),
            (       ' State-gov',),
            (    ' Self-emp-inc',),
            (     ' Federal-gov',),
            (     ' Without-pay',),
            (    ' Never-worked',)],
           names=['workclass'])

In [17]:
df[['workclass']].value_counts()/df.shape[0]*100

# amount of classes in percentage

workclass        
 Private             69.703019
 Self-emp-not-inc     7.803814
 Local-gov            6.427935
 ?                    5.638647
 State-gov            3.986364
 Self-emp-inc         3.427413
 Federal-gov          2.948312
 Without-pay          0.042996
 Never-worked         0.021498
dtype: float64

In [35]:
df1=df[(df['workclass']!='Without-pay')&(df['workclass']!='Never-worked')]
df1.head(2)

age          workclass  fnlwgt   education  education-num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   

        marital-status        occupation    relationship    race    sex  \
0        Never-married      Adm-clerical   Not-in-family   White   Male   
1   Married-civ-spouse   Exec-managerial         Husband   White   Male   

   capital-gain  capital-loss  hours-per-week         country  result  
0          2174             0              40   United-States       1  
1             0             0              13   United-States       1

In [64]:
df1['education'].value_counts()

 HS-grad         10501
 Some-college     7291
 Bachelors        5355
 Masters          1723
 Assoc-voc        1382
 11th             1175
 Assoc-acdm       1067
 10th              933
 7th-8th           646
 Prof-school       576
 9th               514
 12th              433
 Doctorate         413
 5th-6th           333
 1st-4th           168
 Preschool          51
Name: education, dtype: int64

In [65]:
df1['education-num'].value_counts()

9     10501
10     7291
13     5355
14     1723
11     1382
7      1175
12     1067
6       933
4       646
15      576
5       514
8       433
16      413
3       333
2       168
1        51
Name: education-num, dtype: int64

# education and educa-num both are same 

In [67]:
df1=df1.drop('education',axis='columns')
# two columns are same to each other thats why i am taking singel one column

In [68]:
df1.head(2)

age          workclass  fnlwgt  education-num       marital-status  \
0   39          State-gov   77516             13        Never-married   
1   50   Self-emp-not-inc   83311             13   Married-civ-spouse   

         occupation    relationship    race    sex  capital-gain  \
0      Adm-clerical   Not-in-family   White   Male          2174   
1   Exec-managerial         Husband   White   Male             0   

   capital-loss  hours-per-week         country  result  
0             0              40   United-States       1  
1             0              13   United-States       1

In [73]:
df1['marital-status'].value_counts()/df.shape[0]*100
# marital_status in percentage

 Married-civ-spouse       45.993673
 Never-married            32.809189
 Divorced                 13.645158
 Separated                 3.147938
 Widowed                   3.049661
 Married-spouse-absent     1.283744
 Married-AF-spouse         0.070637
Name: marital-status, dtype: float64

In [81]:
df1=pd.get_dummies(df1,columns=['marital-status'])
df1.head(2)
# encoding the marital_status

age          workclass  fnlwgt  education-num        occupation  \
0   39          State-gov   77516             13      Adm-clerical   
1   50   Self-emp-not-inc   83311             13   Exec-managerial   

     relationship    race    sex  capital-gain  capital-loss  hours-per-week  \
0   Not-in-family   White   Male          2174             0              40   
1         Husband   White   Male             0             0              13   

          country  result  marital-status_ Divorced  \
0   United-States       1                         0   
1   United-States       1                         0   

   marital-status_ Married-AF-spouse  marital-status_ Married-civ-spouse  \
0                                  0                                   0   
1                                  0                                   1   

   marital-status_ Married-spouse-absent  marital-status_ Never-married  \
0                                      0                              1   
1                                      0                              0   

   marital-status_ Separated  marital-status_ Widowed  
0                          0                        0  
1                          0                        0

# 'Married-AF-spouse' remove it 

In [83]:
df1=df1.drop('marital-status_ Married-AF-spouse',axis='columns')
df1.head(2)

age          workclass  fnlwgt  education-num        occupation  \
0   39          State-gov   77516             13      Adm-clerical   
1   50   Self-emp-not-inc   83311             13   Exec-managerial   

     relationship    race    sex  capital-gain  capital-loss  hours-per-week  \
0   Not-in-family   White   Male          2174             0              40   
1         Husband   White   Male             0             0              13   

          country  result  marital-status_ Divorced  \
0   United-States       1                         0   
1   United-States       1                         0   

   marital-status_ Married-civ-spouse  marital-status_ Married-spouse-absent  \
0                                   0                                      0   
1                                   1                                      0   

   marital-status_ Never-married  marital-status_ Separated  \
0                              1                          0   
1                              0                          0   

   marital-status_ Widowed  
0                        0  
1                        0

In [85]:
df1=df1.drop('race',axis='columns')
df1.head(2)
# race also removed

age          workclass  fnlwgt  education-num        occupation  \
0   39          State-gov   77516             13      Adm-clerical   
1   50   Self-emp-not-inc   83311             13   Exec-managerial   

     relationship    sex  capital-gain  capital-loss  hours-per-week  \
0   Not-in-family   Male          2174             0              40   
1         Husband   Male             0             0              13   

          country  result  marital-status_ Divorced  \
0   United-States       1                         0   
1   United-States       1                         0   

   marital-status_ Married-civ-spouse  marital-status_ Married-spouse-absent  \
0                                   0                                      0   
1                                   1                                      0   

   marital-status_ Never-married  marital-status_ Separated  \
0                              1                          0   
1                              0                          0   

   marital-status_ Widowed  
0                        0  
1                        0

In [88]:
df1['relationship'].value_counts()

 Husband           13193
 Not-in-family      8305
 Own-child          5068
 Unmarried          3446
 Wife               1568
 Other-relative      981
Name: relationship, dtype: int64

In [90]:
df1=df1.drop('relationship',axis='columns')
df1.head(1)
#relationship also

age   workclass  fnlwgt  education-num     occupation    sex  capital-gain  \
0   39   State-gov   77516             13   Adm-clerical   Male          2174   

   capital-loss  hours-per-week         country  result  \
0             0              40   United-States       1   

   marital-status_ Divorced  marital-status_ Married-civ-spouse  \
0                         0                                   0   

   marital-status_ Married-spouse-absent  marital-status_ Never-married  \
0                                      0                              1   

   marital-status_ Separated  marital-status_ Widowed  
0                          0                        0

In [92]:
len(df1.columns)
# now i have 17 feature

17

In [95]:
df1=pd.get_dummies(data=df1,columns=['sex'])
# encode the sex feature

In [102]:
df1.head(1)

age   workclass  fnlwgt  education-num     occupation  capital-gain  \
0   39   State-gov   77516             13   Adm-clerical          2174   

   capital-loss  hours-per-week         country  result  \
0             0              40   United-States       1   

   marital-status_ Divorced  marital-status_ Married-civ-spouse  \
0                         0                                   0   

   marital-status_ Married-spouse-absent  marital-status_ Never-married  \
0                                      0                              1   

   marital-status_ Separated  marital-status_ Widowed  sex_ Female  sex_ Male  
0                          0                        0            0          1

In [4]:
df['workclass'].value_counts()

 Private             22696
 Self-emp-not-inc     2541
 Local-gov            2093
 ?                    1836
 State-gov            1298
 Self-emp-inc         1116
 Federal-gov           960
 Without-pay            14
 Never-worked            7
Name: workclass, dtype: int64

In [25]:
df.columns

Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week', 'result', 'marital-status_ Divorced',
       'marital-status_ Married-civ-spouse',
       'marital-status_ Married-spouse-absent',
       'marital-status_ Never-married', 'marital-status_ Separated',
       'marital-status_ Widowed', 'sex_ Female', 'sex_ Male', 'workclass_ ?',
       'workclass_ Federal-gov', 'workclass_ Local-gov',
       'workclass_ Never-worked', 'workclass_ Private',
       'workclass_ Self-emp-inc', 'workclass_ Self-emp-not-inc',
       'workclass_ State-gov', 'workclass_ Without-pay', 'occupation_ ?',
       'occupation_ Adm-clerical', 'occupation_ Armed-Forces',
       'occupation_ Craft-repair', 'occupation_ Exec-managerial',
       'occupation_ Farming-fishing', 'occupation_ Handlers-cleaners',
       'occupation_ Machine-op-inspct', 'occupation_ Other-service',
       'occupation_ Priv-house-serv', 'occupation_ Prof-specialty',
       'occupation_ Protective-serv

In [26]:
df.columns=['age', 'salary', 'educ-num', 'capital-gain', 'capital-loss',
       'hrs-pr-week', 'output', 'Divorced',
       ' Married-civ-spouse',
       'Married-spouse-absent',
       ' Never-married', 'Separated',
       'Widowed', ' Female', 'Male', 'other',
       'Federal-gov', 'Local-gov',
       'Never-worked', 'Private',
       'Self-emp-inc', 'Self-emp-not-inc',
       'State-gov', 'Without-pay', 'other_occu',
       'Adm-clerical', 'Armed-Forces',
       'Craft-repair', 'Exec-managerial',
       'Farming-fishing', 'Handlers-cleaners',
       'Machine-op-inspct', 'Other-service_occu',
       'Priv-house-serv', 'Prof-specialty',
       'Protective-serv', 'Sales',
       'Tech-support', 'Transport-moving',
       'other_country', 'Cambodia', 'Canada', 'China',
       'Columbia', 'Cuba', 'Dominican-Republic',
       ' Ecuador', 'El-Salvador', 'England',
       'France', 'Germany', 'Greece',
       'Guatemala', 'Haiti', 'Holand-Netherlands',
       'Honduras', ' Hong', 'Hungary',
       'India', 'Iran', 'Ireland', 'Italy',
       'Jamaica', 'Japan', 'country_ Laos',
       'Mexico', 'Nicaragua',
       'Outlying-US', 'Peru',
       'Philippines', 'Poland', 'Portugal',
       'Puerto-Rico', 'Scotland', 'South',
       'Taiwan', 'country_ Thailand', 'Trinadad&Tobago',
       'United-States', 'Vietnam', 'Yugoslavia']


# rename the columns

In [27]:
df.head(2)

age  salary  educ-num  capital-gain  capital-loss  hrs-pr-week  output  \
0   39   77516        13          2174             0           40       1   
1   50   83311        13             0             0           13       1   

   Divorced   Married-civ-spouse  Married-spouse-absent  ...  Portugal  \
0         0                    0                      0  ...         0   
1         0                    1                      0  ...         0   

   Puerto-Rico  Scotland  South  Taiwan  country_ Thailand  Trinadad&Tobago  \
0            0         0      0       0                  0                0   
1            0         0      0       0                  0                0   

   United-States  Vietnam  Yugoslavia  
0              1        0           0  
1              1        0           0  

[2 rows x 81 columns]

In [33]:
x=df.drop('output',axis='columns')
y=df[['output']]              # define the variable for ML algo.

In [31]:
y.shape,x.shape

((32561,), (32561, 80))

In [34]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=31)     # split the variable into train & test

In [35]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((26048, 80), (6513, 80), (26048, 1), (6513, 1))

In [37]:
rfr=RandomForestClassifier()               # trained the model by using randomforest
rfr.fit(x_train,y_train)

<ipython-input-37-faf1a41341e9>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfr.fit(x_train,y_train)


RandomForestClassifier()

In [38]:
rfr.score(x_test,y_test)         # accuracy of my model by using randomforest

0.8490710885920467

# 84% accuracy

In [40]:
dtr=DecisionTreeClassifier()        # trained the model by using  decisiontree
dtr.fit(x_train,y_train)
dtr.score(x_test,y_test)

#accuracy of my model by using decisiontree

0.8179026562260096

In [41]:
cross_valtidation_sc=cross_val_score(rfr,x_test,y_test)    
cross_valtidation_sc 

# cross validation by randomforest

c:\users\dell\appdata\local\programs\python\python39\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\users\dell\appdata\local\programs\python\python39\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\users\dell\appdata\local\programs\python\python39\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\users\dell\appdata\local\programs\python\py

array([0.85418266, 0.8534152 , 0.85878741, 0.85099846, 0.85483871])

# predict the value by using randomforest

In [43]:
y_pred=rfr.predict(x_test)

In [44]:
rd_mean=mean_squared_error(y_test,y_pred)

In [45]:
rd_mean

0.15092891140795334

In [46]:
np.sqrt(rd_mean)               # root mean squared error

0.38849570320397797

In [47]:
confusion_matrix(y_test,y_pred)              # confusionmetrix

array([[ 980,  602],
       [ 381, 4550]], dtype=int64)

In [48]:
(980+4550)/(980+4550+602+381)

# confision metrix result

0.8490710885920467

In [51]:
y_test['output']           # actual value

24279    0
8364     1
25546    1
6173     0
29227    1
        ..
2066     0
21195    1
30751    1
19584    1
6461     1
Name: output, Length: 6513, dtype: int64

In [53]:
y_pred1=pd.Series(y_pred,name='predicted')
y_pred1           # predicted value

0       0
1       1
2       1
3       0
4       0
       ..
6508    1
6509    1
6510    1
6511    1
6512    1
Name: predicted, Length: 6513, dtype: int64

## comparing with actual value and predicted value

In [58]:
y_test.value_counts()

output
1         4931
0         1582
dtype: int64

In [59]:
y_pred1.value_counts()

1    5152
0    1361
Name: predicted, dtype: int64

# save the model

In [49]:
pickle.dump(rfr,open('model','wb')) 

# 🤘 Thank you iNeuron ✍